In [1]:
import numpy as np
import numpy.ma as ma
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

In [2]:
img_path = r'C:\Users\User\OneDrive - Engenuity (Pvt) Ltd\Github\SPE-testing\Sentinel-hub API\Data\c63cc1c7352bd86d218dbe557a56aa21'
file = 'response.tiff'
file = rxr.open_rasterio(img_path+'/'+file)

In [3]:
print(file.shape)

(4, 2476, 1453)


In [4]:
file

<xarray.DataArray (band: 4, y: 2476, x: 1453)>
[14390512 values with dtype=float32]
Coordinates:
  * band         (band) int32 1 2 3 4
  * x            (x) float64 79.55 79.56 79.56 79.56 ... 81.99 81.99 81.99 81.99
  * y            (y) float64 10.03 10.03 10.03 10.02 ... 5.892 5.89 5.889 5.887
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    scale_factor:            1.0
    add_offset:              0.0

In [5]:
file[3]

<xarray.DataArray (y: 2476, x: 1453)>
[3597628 values with dtype=float32]
Coordinates:
    band         int32 4
  * x            (x) float64 79.55 79.56 79.56 79.56 ... 81.99 81.99 81.99 81.99
  * y            (y) float64 10.03 10.03 10.03 10.02 ... 5.892 5.89 5.889 5.887
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    scale_factor:            1.0
    add_offset:              0.0

In [6]:
ndvi = es.normalized_diff(file[3], file[2])
print(ndvi.shape)
ndvi

(2476, 1453)


masked_array(
  data=[[-0.3661016821861267, -0.36717426776885986, -0.36486485600471497,
         ..., --, --, --],
        [-0.36486485600471497, -0.36593592166900635,
         -0.36717426776885986, ..., --, --, --],
        [-0.36666667461395264, -0.3733333349227905, -0.371048241853714,
         ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32)

In [7]:
ndvi_data = ma.getdata(ndvi)
print(ndvi_data.shape)
ndvi_data

(2476, 1453)


array([[-0.36610168, -0.36717427, -0.36486486, ...,         nan,
                nan,         nan],
       [-0.36486486, -0.36593592, -0.36717427, ...,         nan,
                nan,         nan],
       [-0.36666667, -0.37333333, -0.37104824, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]], dtype=float32)

In [8]:
arr = ndvi_data.copy()
arr

array([[-0.36610168, -0.36717427, -0.36486486, ...,         nan,
                nan,         nan],
       [-0.36486486, -0.36593592, -0.36717427, ...,         nan,
                nan,         nan],
       [-0.36666667, -0.37333333, -0.37104824, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]], dtype=float32)

In [9]:
rows, cols = arr.shape
print(rows)
print(cols)

2476
1453


In [10]:
arr[2475][1452]

nan

In [15]:
for i in range(0,rows-1):
    for j in range(0,cols-1):
        if (ndvi_data[i][j] >= 0.6 and ndvi_data[i][j] <= 0.8):
            arr[i][j] = 1
        else:
            arr[i][j] = 0

In [16]:
arr

array([[ 0.,  0.,  0., ...,  0.,  0., nan],
       [ 0.,  0.,  0., ...,  0.,  0., nan],
       [ 0.,  0.,  0., ...,  0.,  0., nan],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0., nan],
       [ 0.,  0.,  0., ...,  0.,  0., nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [17]:
arr_list = arr.tolist()
green_pixel = sum(x.count(1) for x in arr_list)
green_pixel

1124761

In [18]:
res = 185
green_area = (green_pixel*res*res)/1000000 # in km2
print(green_area, "sqr kilometers")

38494.945225 sqr kilometers


In [19]:
total_area = 65610 # in km2
green_cover = np.round(green_area/total_area*100,2)
print("green_cover: ",green_cover,"%")

green_cover:  58.67 %


In [20]:
def green_area(ndvi,res,lw_thresh=0.6,up_thresh=0.8):
    
    '''
    Calculate the green area with specific threshold values
    
    ndvi: ndvi array
    res: image resolution
    thresh: threshold value to distinguish forect/ green area
    
    return: green cover in km2
    '''
    
    # get the ndvi data from masked array
    ndvi_data = ma.getdata(ndvi)
    arr = ndvi_data.copy()
    
    # get the row & col count
    rows, cols = arr.shape
    
    # replace ndvi values with 1, if ndvi >= threshold value
    for i in range(0,rows-1):
        for j in range(0,cols-1):
            if (ndvi_data[i][j] >= lw_thresh and ndvi_data[i][j] <= up_thresh):
                arr[i][j] = 1
            else:
                arr[i][j] = 0
    
    # count the ones (green pixel)
    arr_list = arr.tolist()
    green_pixel = sum(x.count(1) for x in arr_list)
    
    # calculate the green area in km2
    green_area = (green_pixel*res*res)/1000000 # in km2
    print(green_area, "sqr kilometers")
    
    return green_area

In [22]:
ndvi = es.normalized_diff(file[3], file[2])
green_area(ndvi,185)

38494.945225 sqr kilometers


38494.945225

## Function

In [149]:
def green_area(ndvi,res,thresh=0.71):
    
    '''
    Calculate the green area with specific threshold values
    
    ndvi: ndvi array
    res: image resolution
    thresh: threshold value to distinguish forect/ green area
    
    return: green cover in km2
    '''
    
    # get the ndvi data from masked array
    ndvi_data = ma.getdata(ndvi)
    # get the row & col count
    arr = ndvi_data.copy()
    rows, cols = arr.shape
    
    # replace ndvi values with 1, if ndvi >= threshold value
    for i in range(0,rows-1):
        for j in range(0,cols-1):
            if arr[i][j] >= thresh:
                arr[i][j] = 1
            else:
                arr[i][j] = 0
            
    # count the ones
    ndvi_list = arr.tolist()
    green_pixel = sum(x.count(1) for x in ndvi_list)
    
    # calculate the green area in km2
    green_area = (green_pixel*res*res)/1000000
    
    return green_area

In [150]:
# calculate forest/ green cover percentage with user input total area

def green_cover(green_area,total_area):
    green_cover = np.round((green_area/total_area*100),2)
    print("Forest/green cover: ",green_cover,"%")

In [152]:
ndvi = es.normalized_diff(file[3], file[2])
green = green_area(ndvi,185)
green_cover(green,65610)

Forest/green cover:  28.39 %
